<a href="https://colab.research.google.com/github/pavi-ninjaac/Specrom_Analysis/blob/main/Internship_works/week3/text_classification_109.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
#get the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

#sklearn functions
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

#sklearn vectorization packages
from sklearn.feature_extraction.text import TfidfVectorizer

#sklearn models
from sklearn.linear_model import LogisticRegression

#sklearn evaluation metrics
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataFiles/Text_classification_news_article/rishika_csv_different_computer_with_iptc.csv',index_col=0)

shape = data.shape
columns = list(data.columns)

print("the shape of the dataset is--->",shape)
print('The features are-------------->',columns)

data.head(4)


the shape of the dataset is---> (10761, 8)
The features are--------------> ['position', 'query', 'url', 'title', 'snippet', 'combined_text', 'level1_labels', 'level2_labels']


,position,query,url,title,snippet,combined_text,level1_labels,level2_labels
0,1,"arts, culture, entertainment and media",https://www.bbc.com/news/entertainment_and_arts,Entertainment & Arts - BBC News,Get the latest BBC Entertainment and Arts news...,Entertainment & Arts - BBC News Get the latest...,"arts, culture, entertainment and media",NaN
1,2,"arts, culture, entertainment and media",https://www.champaigncounty.org/list/ql/arts-c...,"Arts, Culture & Entertainment | Champaign Coun...",Champaign County Chamber of Commerce - IL 303 ...,"Arts, Culture & Entertainment | Champaign Coun...","arts, culture, entertainment and media",NaN
2,3,"arts, culture, entertainment and media",https://www.bls.gov/oes/current/oes270000.htm,"Arts, Design, Entertainment, Sports, and Media...","27-0000 Arts, Design, Entertainment, Sports, a...","Arts, Design, Entertainment, Sports, and Media...","arts, culture, entertainment and media",NaN
3,4,"arts, culture, entertainment and media",https://www.smc.edu/academics/areas-of-interes...,"Arts, Media, & Entertainment - Santa Monica Co...",Let your imagination run wild and free — explo...,"Arts, Media, & Entertainment - Santa Monica Co...","arts, culture, entertainment and media",NaN


In [3]:
print("The nunber of category----------->",data['level1_labels'].nunique())
print("The nunber of second category---->",data['level2_labels'].nunique())

The nunber of category-----------> 17
The nunber of second category----> 109


In [4]:
print("The nunber of quieries---->",data['query'].nunique())

The nunber of quieries----> 1296


In [5]:
data['position'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [6]:
needed_columns = ['combined_text','query','level1_labels','level2_labels']
data = data[needed_columns].copy()
data.head(4)

,combined_text,query,level1_labels,level2_labels
0,Entertainment & Arts - BBC News Get the latest...,"arts, culture, entertainment and media","arts, culture, entertainment and media",NaN
1,"Arts, Culture & Entertainment | Champaign Coun...","arts, culture, entertainment and media","arts, culture, entertainment and media",NaN
2,"Arts, Design, Entertainment, Sports, and Media...","arts, culture, entertainment and media","arts, culture, entertainment and media",NaN
3,"Arts, Media, & Entertainment - Santa Monica Co...","arts, culture, entertainment and media","arts, culture, entertainment and media",NaN


# Handling dublicate values

In [7]:
#delete the dublicate values
data.duplicated().sum() # count the total duplicate samples

362

In [8]:
data.drop_duplicates(keep='last',inplace=True)

In [9]:

#there can be dublicate of author names so check for the dublicate headline and short discription
data.duplicated(subset=[ 'combined_text']).sum()

751

In [10]:
data.drop_duplicates(subset='combined_text' , keep ='last' , inplace=True)

In [11]:
print("THe length of the datset after dublicate deletion------>",data.shape)

THe length of the datset after dublicate deletion------> (9648, 4)


# Handling null values

In [12]:
data.isnull().sum()

combined_text      1
query              0
level1_labels      0
level2_labels    133
dtype: int64

In [13]:
data.dropna(axis=0 ,inplace=True , how='any')

In [14]:
print("THe length of the datset after null values deletion------>",data.shape)

THe length of the datset after null values deletion------> (9514, 4)


In [15]:
data[data['combined_text'] == ' ']
data = data.reset_index()
data = data.drop('index',axis=1)
data.head(4)

,combined_text,query,level1_labels,level2_labels
0,A&E | Watch Full Episodes of Your Favorite Sho...,arts and entertainment,"arts, culture, entertainment and media",arts and entertainment
1,A&E TV Shows | A&E Check out A&E's shows lineu...,arts and entertainment,"arts, culture, entertainment and media",arts and entertainment
2,Entertainment & Arts - BBC News Get the latest...,arts and entertainment,"arts, culture, entertainment and media",arts and entertainment
3,Arts & Entertainment | news-gazette.com · Clo...,arts and entertainment,"arts, culture, entertainment and media",arts and entertainment


# Cleaning the combined text

In [16]:
def preprocessor_text(text):
    if isinstance((text), (str)):
        text = re.sub('<[^>]*>', ' ', text)
        text = re.sub('[\W]+', ' ', text.lower())
        return text
    if isinstance((text), (list)):
        return_list = []
        for i in range(len(text)):
            temp_text = re.sub('<[^>]*>', '', text[i])
            temp_text = re.sub('[\W]+', '', temp_text.lower())
            return_list.append(temp_text)
        return(return_list)

data['combined_text'] = data['combined_text'].apply(preprocessor_text)

data.head(4)

,combined_text,query,level1_labels,level2_labels
0,a e watch full episodes of your favorite shows...,arts and entertainment,"arts, culture, entertainment and media",arts and entertainment
1,a e tv shows a e check out a e s shows lineup ...,arts and entertainment,"arts, culture, entertainment and media",arts and entertainment
2,entertainment arts bbc news get the latest bbc...,arts and entertainment,"arts, culture, entertainment and media",arts and entertainment
3,arts entertainment news gazette com cloudy sno...,arts and entertainment,"arts, culture, entertainment and media",arts and entertainment


# Balance the dataset

In [17]:
#helper functions
def downsample(df , label):
  g = df.groupby(label , group_keys=False)
  balaced_df = pd.DataFrame(g.apply(lambda x : x.sample(g.size().min()))).reset_index(drop=True)
  return balaced_df


def balace_category(data , category_column , downsample_value):
  #to handle the categories easily, we have to change the dtype of the 'query' to category
  data[category_column] = data[category_column].astype('category')

  category = data[category_column].value_counts().index
  print("The shape before deletion------------------------------------------>",data.shape)
  print()
  print("The number of category in the {} column------------------------->{}".format(category_column , len(category)))

  #seperate the column which has to be delete
  need_to_be_deleted = category[(data[category_column].value_counts() < downsample_value)]
  print("The number of category need to be deleted in the {} column------>{}".format(category_column , len(need_to_be_deleted)))

  #delete the unwanted category
  data[category_column] = data[category_column].cat.remove_categories(need_to_be_deleted) #set nan for the categories need to be deletd and then use dropna to delete those things
  data.dropna(axis=0 , how='any',inplace=True) #delete the nan category

  print("The number of category after deletion------------------------------>",len(data[category_column].value_counts().index))
  print()
  print("The shape after deletion------------------------------------------->",data.shape)
  print()

  #down sample the query category to its minimum value
  data = downsample(data , category_column) #call the helper function
  #data[category_column].value_counts()

  #saffule the dataset
  data = shuffle(data)
  data.reset_index(inplace=True, drop=True) 
  

  return data

In [18]:
#for my understanding- if you cant understand the healper function , uncommend this and try each parts seperately, you will get a clear idea of what is happening
"""
#part-1
#to handle the categories easily, we have to change the dtype of the 'query' to category
data_query['query'] = data_query['query'].astype('category')

#part-2
query_category = data_query['query'].value_counts().index
print("The shape before deletion------------------------------------------>",data_query.shape)
print()
print("The number of category in the query column------------------------->",len(query_category))

#seperate the column which has to be delete
need_to_be_deleted = query_category[(data_query['query'].value_counts()<6)]
print("The number of category need to be deleted in the query column------>",len(need_to_be_deleted))

#delete the unwanted category
data_query['query'] = data_query['query'].cat.remove_categories(need_to_be_deleted) #set nan for the categories need to be deletd and then use dropna to delete those things
data_query.dropna(axis=0 , how='any',inplace=True) #delete the nan category

print("The number of category after deletion------------------------------>",len(data_query['query'].value_counts().index))
print()
print("The shape after deletion------------------------------------------->",data_query.shape)
print()

#part-3
#down sample the query category to its minimum value
data_query = downsample(data_query , 'query')
data_query['query'].value_counts()"""


'\n#part-1\n#to handle the categories easily, we have to change the dtype of the \'query\' to category\ndata_query[\'query\'] = data_query[\'query\'].astype(\'category\')\n\n#part-2\nquery_category = data_query[\'query\'].value_counts().index\nprint("The shape before deletion------------------------------------------>",data_query.shape)\nprint()\nprint("The number of category in the query column------------------------->",len(query_category))\n\n#seperate the column which has to be delete\nneed_to_be_deleted = query_category[(data_query[\'query\'].value_counts()<6)]\nprint("The number of category need to be deleted in the query column------>",len(need_to_be_deleted))\n\n#delete the unwanted category\ndata_query[\'query\'] = data_query[\'query\'].cat.remove_categories(need_to_be_deleted) #set nan for the categories need to be deletd and then use dropna to delete those things\ndata_query.dropna(axis=0 , how=\'any\',inplace=True) #delete the nan category\n\nprint("The number of category a

In [25]:
#healper function
label = LabelEncoder()

def split_and_ifidfVectorization(X , y , max_df , min_df):

  #label the y value
  y = label.fit_transform(y)
  #split the dataset
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

  #tfidf
  vecto =  TfidfVectorizer(stop_words='english',max_df = max_df ,min_df= min_df,
                                   ngram_range=(1, 2),lowercase=True, max_features=5000)
  vecto = vecto.fit(X_train)

  X_train = vecto.transform(X_train).toarray()
  X_test = vecto.transform(X_test).toarray()

  #print the shapes
  print("The shape of the X train----------->",X_train.shape)
  print("The shape of the y train----------->",y_train.shape)
  print("The shape of the X test------------>",X_test.shape)
  print("The shape of the y test------------>",y_test.shape)

  return X_train,X_test,y_train,y_test,vecto

# Model training

In [32]:
def classifier_scores(y_train,y_test, pred_train, pred_test):
    
    print()
    print("Train data accuracy score: ", accuracy_score(y_train,pred_train))    
    print("Test data accuracy score: ", accuracy_score(y_test,pred_test))
    print()
    print("Recall score on train data: ", recall_score(y_train,pred_train, average='macro'))
    print("Recall score on test data: ",recall_score( y_test,pred_test, average='macro'))
    print()
    
    print("Precision score on train data: ",precision_score(y_train,pred_train, average='macro'))
    print("Precision score on test data: ",precision_score(y_test,pred_test, average='macro'))
    print()
    print("F1 score on train data: ",f1_score(y_train,pred_train, average='macro'))
    print("F1 score on test data: ",f1_score(y_test,pred_test, average='macro'))

# Logistric Regression

In [33]:
def logistricRegression( X_train,X_test,y_train,y_test):
  log_reg = LogisticRegression()

  log_reg.fit(X_train , y_train)

  y_train_pred = log_reg.predict(X_train)
  y_test_pred = log_reg.predict(X_test)

  classifier_scores(y_train,y_test,y_train_pred,y_test_pred) #call the healper function

  return log_reg

### 'query' categories

- When doing the balancing , we have to delete the categorie, which are having very few sample. Here by i am deleting the categories which are having less than 6 sample. 
- after deletion, we have to resample the data by select 6 samples randomly from each categories. 

In [23]:
#copy the original data 
data2 = data.copy()
data_query = data[['combined_text' , 'query']].copy()

#balace the dataset according to 'query' column
data_query = balace_category(data = data_query , category_column= 'query' , downsample_value= 6)

The shape before deletion------------------------------------------> (9514, 2)

The number of category in the query column------------------------->1279
The number of category need to be deleted in the query column------>252
The number of category after deletion------------------------------> 1027

The shape after deletion-------------------------------------------> (8446, 2)



In [31]:
X = data_query['combined_text']
y = data_query['query']

X_train,X_test,y_train,y_test,vecto_query = split_and_ifidfVectorization(X,y , max_df = 0.99 , min_df = 6)

The shape of the X train-----------> (4929, 4546)
The shape of the y train-----------> (4929,)
The shape of the X test------------> (1233, 4546)
The shape of the y test------------> (1233,)


In [35]:
# train the logistric regression model
log_reg_query = logistricRegression(X_train,X_test,y_train,y_test)


Train data accuracy score:  0.8967336173666058
Test data accuracy score:  0.35279805352798055

Recall score on train data:  0.8642810775722168
Recall score on test data:  0.35648148148148145

Precision score on train data:  0.8899904542066177
Precision score on test data:  0.32965633903133906

F1 score on train data:  0.8594182192967016
F1 score on test data:  0.32100461267127933


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Level1_label classification

In [36]:
# set the dataset
data_lavel1 = data[['combined_text' , 'level1_labels']]

#check fot the downsample value
data_lavel1['level1_labels'].value_counts()

sport                                        2416
economy, business and finance                1940
politics                                      757
arts, culture, entertainment and media        536
crime, law and justice                        477
health                                        452
science and technology                        427
religion and belief                           422
society                                       403
lifestyle and leisure                         396
labour                                        259
disaster, accident and emergency incident     237
conflict, war and peace                       221
environment                                   215
education                                     188
human interest                                135
weather                                        33
Name: level1_labels, dtype: int64